In [13]:
import pandas as pd
import numpy as np

orders = pd.read_csv("orders.csv")
delivery = pd.read_csv("delivery_performance.csv")
routes = pd.read_csv("routes_distance.csv")
vehicle = pd.read_csv("vehicle_fleet.csv")

print("Orders columns:", list(orders.columns))
print("Delivery columns:", list(delivery.columns))
print("Routes columns:", list(routes.columns))
print("Vehicle columns:", list(vehicle.columns))
print("\n--- Delivery sample ---")
display(delivery.head())

for col in ['Promised_Delivery_Days', 'Actual_Delivery_Days']:
    if col in delivery.columns:
        delivery[col] = pd.to_numeric(delivery[col], errors='coerce')
    else:
        print(f"WARNING: column {col} not found in delivery. Please tell me the exact name printed above.")


if ('Promised_Delivery_Days' in delivery.columns) and ('Actual_Delivery_Days' in delivery.columns):
    delivery['Delay_Days'] = delivery['Actual_Delivery_Days'] - delivery['Promised_Delivery_Days']
    delivery['Delay_Days'] = delivery['Delay_Days'].fillna(0)
else:
   
    raise KeyError("Promised/Actual delivery columns missing — check the 'Delivery columns' printed above and tell me the exact names.")


threshold_days = 0
delivery['Delay_Status_auto'] = delivery['Delay_Days'].apply(lambda x: 'Delayed' if x > threshold_days else 'On Time')


print("\nPreview of delay columns:")
display(delivery[['Order_ID','Promised_Delivery_Days','Actual_Delivery_Days','Delay_Days','Delay_Status_auto']].head(10))

print("\nCounts of Delay_Status_auto:")
print(delivery['Delay_Status_auto'].value_counts())

Orders columns: ['Order_ID', 'Order_Date', 'Customer_Segment', 'Priority', 'Product_Category', 'Order_Value_INR', 'Origin', 'Destination', 'Special_Handling']
Delivery columns: ['Order_ID', 'Carrier', 'Promised_Delivery_Days', 'Actual_Delivery_Days', 'Delivery_Status', 'Quality_Issue', 'Customer_Rating', 'Delivery_Cost_INR']
Routes columns: ['Order_ID', 'Route', 'Distance_KM', 'Fuel_Consumption_L', 'Toll_Charges_INR', 'Traffic_Delay_Minutes', 'Weather_Impact']
Vehicle columns: ['Vehicle_ID', 'Vehicle_Type', 'Capacity_KG', 'Fuel_Efficiency_KM_per_L', 'Current_Location', 'Status', 'Age_Years', 'CO2_Emissions_Kg_per_KM']

--- Delivery sample ---


,Order_ID,Carrier,Promised_Delivery_Days,Actual_Delivery_Days,Delivery_Status,Quality_Issue,Customer_Rating,Delivery_Cost_INR
0,ORD000001,SpeedyLogistics,1,2,Slightly-Delayed,Perfect,3,387.86
1,ORD000002,SpeedyLogistics,2,3,Slightly-Delayed,Minor_Damage,1,430.19
2,ORD000003,SpeedyLogistics,10,15,Severely-Delayed,Minor_Damage,3,1039.19
3,ORD000004,QuickShip,5,5,On-Time,Perfect,5,599.01
4,ORD000005,SpeedyLogistics,4,5,Slightly-Delayed,Perfect,3,537.90



Preview of delay columns:


,Order_ID,Promised_Delivery_Days,Actual_Delivery_Days,Delay_Days,Delay_Status_auto
0,ORD000001,1,2,1,Delayed
1,ORD000002,2,3,1,Delayed
2,ORD000003,10,15,5,Delayed
3,ORD000004,5,5,0,On Time
4,ORD000005,4,5,1,Delayed
5,ORD000006,1,2,1,Delayed
6,ORD000007,3,3,0,On Time
7,ORD000008,5,10,5,Delayed
8,ORD000009,2,2,0,On Time
9,ORD000010,4,9,5,Delayed



Counts of Delay_Status_auto:
Delay_Status_auto
On Time    80
Delayed    70
Name: count, dtype: int64


In [14]:
import pandas as pd

orders = pd.read_csv("orders.csv")
delivery = pd.read_csv("delivery_performance.csv")
routes = pd.read_csv("routes_distance.csv")
vehicle = pd.read_csv("vehicle_fleet.csv")

print("Orders columns:", list(orders.columns))
print("Delivery columns:", list(delivery.columns))
print("Routes columns:", list(routes.columns))
print("Vehicle columns:", list(vehicle.columns))

# Common approach: orders and delivery can join on 'Order_ID'
# routes might also have Order_ID; vehicles may have 'Vehicle_ID'
left_key = 'Order_ID' if 'Order_ID' in orders.columns and 'Order_ID' in delivery.columns else None
print("\nChosen orders<->delivery key:", left_key)

# If routes has Order_ID join on Order_ID else try 'Route_ID' or origin/destination
route_key = 'Order_ID' if 'Order_ID' in routes.columns else ('Route_ID' if 'Route_ID' in routes.columns else None)
print("Chosen delivery<->routes key:", route_key)

# Vehicle join key (common): 'Vehicle_ID' present in delivery or routes?
veh_key = None
if 'Vehicle_ID' in vehicle.columns:
    if 'Vehicle_ID' in delivery.columns:
        veh_key = 'Vehicle_ID'   # delivery has vehicle assignment
    elif 'Vehicle_ID' in routes.columns:
        veh_key = 'Vehicle_ID'   # or routes
print("Chosen vehicle key:", veh_key)

master = orders.copy()

if left_key:
    master = master.merge(delivery, on=left_key, how='left', suffixes=('','_del'))
else:
    
    print("WARNING: No Order_ID found in both orders & delivery. Performing concat-by-index fallback.")
    master = pd.concat([orders, delivery], axis=1)

if route_key:
    master = master.merge(routes, on=route_key, how='left', suffixes=('','_route'))
else:
    print("Note: no route key found - skipping routes merge or you may merge on origin/destination manually later.")

if veh_key:
    master = master.merge(vehicle, on=veh_key, how='left', suffixes=('','_veh'))
else:
    print("Note: no vehicle key found to merge - skipping vehicle merge.")

print("\nMASTER shape:", master.shape)
print("\nColumns in MASTER:", list(master.columns))

# show missing percent for important columns
missing = master.isna().sum().sort_values(ascending=False)
missing_percent = (missing / len(master) * 100).round(2)
print("\nTop missing columns (count, %):")
print(pd.concat([missing, missing_percent], axis=1).rename(columns={0:'missing_count',1:'missing_pct'}).head(20))

display(master.head(12))

# Save master to CSV so you can use it later in modeling or the Streamlit app
master.to_csv("master_logistics_table.csv", index=False)
print("\nSaved master_logistics_table.csv in project folder.")


Orders columns: ['Order_ID', 'Order_Date', 'Customer_Segment', 'Priority', 'Product_Category', 'Order_Value_INR', 'Origin', 'Destination', 'Special_Handling']
Delivery columns: ['Order_ID', 'Carrier', 'Promised_Delivery_Days', 'Actual_Delivery_Days', 'Delivery_Status', 'Quality_Issue', 'Customer_Rating', 'Delivery_Cost_INR']
Routes columns: ['Order_ID', 'Route', 'Distance_KM', 'Fuel_Consumption_L', 'Toll_Charges_INR', 'Traffic_Delay_Minutes', 'Weather_Impact']
Vehicle columns: ['Vehicle_ID', 'Vehicle_Type', 'Capacity_KG', 'Fuel_Efficiency_KM_per_L', 'Current_Location', 'Status', 'Age_Years', 'CO2_Emissions_Kg_per_KM']

Chosen orders<->delivery key: Order_ID
Chosen delivery<->routes key: Order_ID
Chosen vehicle key: None
Note: no vehicle key found to merge - skipping vehicle merge.

MASTER shape: (200, 22)

Columns in MASTER: ['Order_ID', 'Order_Date', 'Customer_Segment', 'Priority', 'Product_Category', 'Order_Value_INR', 'Origin', 'Destination', 'Special_Handling', 'Carrier', 'Promised

,Order_ID,Order_Date,Customer_Segment,Priority,Product_Category,Order_Value_INR,Origin,Destination,Special_Handling,Carrier,...,Delivery_Status,Quality_Issue,Customer_Rating,Delivery_Cost_INR,Route,Distance_KM,Fuel_Consumption_L,Toll_Charges_INR,Traffic_Delay_Minutes,Weather_Impact
0,ORD000001,2025-10-09,Individual,Express,Industrial,238.73,Kolkata,Hyderabad,NaN,SpeedyLogistics,...,Slightly-Delayed,Perfect,3.0,387.86,Kolkata-Hyderabad,152.59,23.02,122.08,21.0,NaN
1,ORD000002,2025-09-29,SMB,Express,Industrial,17.01,Hyderabad,Kolkata,NaN,SpeedyLogistics,...,Slightly-Delayed,Minor_Damage,1.0,430.19,Hyderabad-Kolkata,362.05,43.98,289.64,33.0,NaN
2,ORD000003,2025-09-15,SMB,Economy,Industrial,3024.95,Mumbai,Pune,NaN,SpeedyLogistics,...,Severely-Delayed,Minor_Damage,3.0,1039.19,Mumbai-Pune,519.74,65.75,415.79,2.0,NaN
3,ORD000004,2025-10-13,Individual,Economy,Fashion,56.74,Hyderabad,Ahmedabad,NaN,QuickShip,...,On-Time,Perfect,5.0,599.01,Hyderabad-Ahmedabad,540.87,61.85,432.70,112.0,NaN
4,ORD000005,2025-09-08,SMB,Standard,Fashion,19148.65,Chennai,Mumbai,NaN,SpeedyLogistics,...,Slightly-Delayed,Perfect,3.0,537.90,Chennai-Mumbai,1251.56,147.54,1001.25,10.0,NaN
5,ORD000006,2025-09-21,Individual,Express,Food & Beverage,158.22,Chennai,Dubai,NaN,GlobalTransit,...,Slightly-Delayed,Perfect,4.0,378.99,Chennai-Dubai,3041.83,365.82,0.00,0.0,NaN
6,ORD000007,2025-10-09,Individual,Standard,Electronics,736.82,Pune,Bangalore,NaN,QuickShip,...,On-Time,Perfect,4.0,439.72,Pune-Bangalore,854.87,103.85,683.90,10.0,NaN
7,ORD000008,2025-09-19,Enterprise,Standard,Industrial,4771.54,Chennai,Kolkata,NaN,GlobalTransit,...,Severely-Delayed,Perfect,2.0,836.35,Chennai-Kolkata,1430.06,173.95,1144.05,81.0,NaN
8,ORD000009,2025-09-23,SMB,Express,Books,3711.35,Delhi,Hong Kong,NaN,SpeedyLogistics,...,On-Time,Perfect,5.0,414.80,Delhi-Hong Kong,3708.13,446.18,0.00,0.0,NaN
9,ORD000010,2025-09-11,SMB,Standard,Electronics,7936.32,Hyderabad,Mumbai,Hazmat,GlobalTransit,...,Severely-Delayed,Minor_Damage,3.0,891.65,Hyderabad-Mumbai,771.73,88.00,617.39,16.0,Light_Rain



Saved master_logistics_table.csv in project folder.


In [15]:

import pandas as pd

# Load the master dataset (just to be safe)
master = pd.read_csv("master_logistics_table.csv")

# 1. Check missing data percentage
missing = master.isna().sum().sort_values(ascending=False)
missing_percent = (missing / len(master) * 100).round(2)
missing_table = pd.concat([missing, missing_percent], axis=1).rename(columns={0:'Missing_Count',1:'Missing_%'})
print("Missing values summary (top 15):\n")
print(missing_table.head(15))

# 2. Drop columns with too many missing values (>50%)
threshold = 50
cols_to_drop = missing_table[missing_table['Missing_%'] > threshold].index.tolist()
print(f"\nDropping columns with >{threshold}% missing:", cols_to_drop)
master = master.drop(columns=cols_to_drop)

# 3. Fill remaining missing values with 0 or 'Unknown'
for col in master.select_dtypes(include='number'):
    master[col] = master[col].fillna(0)
for col in master.select_dtypes(include='object'):
    master[col] = master[col].fillna('Unknown')

# 4. Check final missing data
print("\nMissing values after cleaning:")
print(master.isna().sum().sum(), "missing values left")

# 5. Select feature columns and target
# We'll predict Delay_Status_auto (from delivery)
target_col = 'Delay_Status_auto'
features = [col for col in master.columns if col != target_col]

print("\nFeature columns for model:")
print(features)

# Save cleaned dataset for modeling
master.to_csv("cleaned_master_dataset.csv", index=False)
print("\n✅ Cleaned dataset saved as 'cleaned_master_dataset.csv'")


Missing values summary (top 15):

                        Missing_Count  Missing_%
Weather_Impact                    156       78.0
Special_Handling                  153       76.5
Carrier                            50       25.0
Actual_Delivery_Days               50       25.0
Quality_Issue                      50       25.0
Delivery_Status                    50       25.0
Distance_KM                        50       25.0
Fuel_Consumption_L                 50       25.0
Route                              50       25.0
Delivery_Cost_INR                  50       25.0
Toll_Charges_INR                   50       25.0
Traffic_Delay_Minutes              50       25.0
Customer_Rating                    50       25.0
Promised_Delivery_Days             50       25.0
Order_Value_INR                     0        0.0

Dropping columns with >50% missing: ['Weather_Impact', 'Special_Handling']

Missing values after cleaning:
0 missing values left

Feature columns for model:
['Order_ID', 'Order_Date

In [18]:
import pandas as pd

data = pd.read_csv("cleaned_master_dataset.csv")
print("Columns in your dataset:\n")
print(list(data.columns))


Columns in your dataset:

['Order_ID', 'Order_Date', 'Customer_Segment', 'Priority', 'Product_Category', 'Order_Value_INR', 'Origin', 'Destination', 'Carrier', 'Promised_Delivery_Days', 'Actual_Delivery_Days', 'Delivery_Status', 'Quality_Issue', 'Customer_Rating', 'Delivery_Cost_INR', 'Route', 'Distance_KM', 'Fuel_Consumption_L', 'Toll_Charges_INR', 'Traffic_Delay_Minutes']


In [20]:
# Robust training block - copy & run the whole cell
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 1) load cleaned data
data = pd.read_csv("cleaned_master_dataset.csv")
print("Columns in dataset:\n")
print(list(data.columns))
print("\nRows:", len(data))

# 2) pick target automatically from common names (first match)
possible_targets = ['Delay_Status_auto', 'Delivery_Status', 'Delay_Status', 'Delay Status', 'delay_status_auto']
target_col = None
for t in possible_targets:
    if t in data.columns:
        target_col = t
        break

if target_col is None:
    raise KeyError("No target column found automatically. Please tell me which column is the target (send me the printed column list).")

print("\nUsing target column:", target_col)

# 3) drop the target safely and also drop obvious ID/date columns which are not features
X = data.drop(columns=[target_col, 'Order_ID', 'Order_Date'] , errors='ignore')  # errors='ignore' avoids KeyError
y = data[target_col].copy()

# 4) quick check: no empty X
if X.shape[1] == 0:
    raise ValueError("No feature columns left after dropping non-features. Check column list.")

# 5) encode categorical columns in X with LabelEncoder (simple approach)
X_enc = X.copy()
label_encoders = {}
for col in X_enc.columns:
    if X_enc[col].dtype == 'object' or X_enc[col].dtype.name == 'category':
        le = LabelEncoder()
        X_enc[col] = le.fit_transform(X_enc[col].astype(str))
        label_encoders[col] = le

# 6) encode target y
le_y = LabelEncoder()
y_enc = le_y.fit_transform(y.astype(str))
print("\nTarget classes (encoded):", dict(zip(le_y.classes_, range(len(le_y.classes_)))))

# 7) split
X_train, X_test, y_train, y_test = train_test_split(X_enc, y_enc, test_size=0.2, random_state=42, stratify=y_enc)

# 8) train RandomForest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# 9) predict & evaluate
y_pred = rf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {acc*100:.2f}%")
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification report:\n", classification_report(y_test, y_pred))

# 10) feature importances
importances = pd.Series(rf.feature_importances_, index=X_enc.columns).sort_values(ascending=False)
print("\nTop 10 feature importances:")
print(importances.head(10))

# 11) save model artifacts if you want (optional)
# import joblib
# joblib.dump(rf, "rf_model.joblib")
# joblib.dump(le_y, "target_label_encoder.joblib")
# print("Saved model and label encoder.")


Columns in dataset:

['Order_ID', 'Order_Date', 'Customer_Segment', 'Priority', 'Product_Category', 'Order_Value_INR', 'Origin', 'Destination', 'Carrier', 'Promised_Delivery_Days', 'Actual_Delivery_Days', 'Delivery_Status', 'Quality_Issue', 'Customer_Rating', 'Delivery_Cost_INR', 'Route', 'Distance_KM', 'Fuel_Consumption_L', 'Toll_Charges_INR', 'Traffic_Delay_Minutes']

Rows: 200

Using target column: Delivery_Status

Target classes (encoded): {'On-Time': 0, 'Severely-Delayed': 1, 'Slightly-Delayed': 2, 'Unknown': 3}

Accuracy: 92.50%

Confusion matrix:
 [[16  0  0  0]
 [ 0  4  1  0]
 [ 2  0  7  0]
 [ 0  0  0 10]]

Classification report:
               precision    recall  f1-score   support

           0       0.89      1.00      0.94        16
           1       1.00      0.80      0.89         5
           2       0.88      0.78      0.82         9
           3       1.00      1.00      1.00        10

    accuracy                           0.93        40
   macro avg       0.94    

In [21]:
# Save model + target label encoder (run this in the same kernel after training)
import joblib

# rf is your trained RandomForest, le_y is the LabelEncoder for the target (from the training cell)
joblib.dump(rf, "rf_model.joblib")
joblib.dump(le_y, "target_label_encoder.joblib")
print("Saved rf_model.joblib and target_label_encoder.joblib")


Saved rf_model.joblib and target_label_encoder.joblib
